In [1]:
import npnet as tn
import ocr_data

import numpy as np

/home/benland100/Desktop/npnet/npnet.py:23: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if init is 'none':
/home/benland100/Desktop/npnet/npnet.py:25: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif init is 'xavier':
/home/benland100/Desktop/npnet/npnet.py:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif init is 'he':


In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = [10,7]
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['xtick.minor.visible'] = True
matplotlib.rcParams['ytick.right'] = True
matplotlib.rcParams['ytick.direction'] = 'in'
matplotlib.rcParams['ytick.minor.visible'] = True
matplotlib.rcParams['font.size'] = 19
matplotlib.rcParams['font.family'] = 'DejaVu Serif'
matplotlib.rcParams['mathtext.default'] = 'regular'
matplotlib.rcParams['errorbar.capsize'] = 3

In [3]:
input_shape = ocr_data.in_2d_shape
conv_kernels = [(5,5)] #[(3,3),(3,3),(3,3),(3,3),(3,3)]
conv_stride = [(3,3)] #[(1,1),(2,2),(1,1),(2,2),(1,1)]
conv_outs = [(4,)] #[(4,),(8,),(8,),(16,),(16,)]
conv_layers = len(conv_kernels)
hidden_shapes = []
hidden_layers = len(hidden_shapes)
output_shape = ocr_data.out_shape
print(input_shape,output_shape)

(32, 32) (26,)


In [4]:
in_layer = tn.Input(input_shape)()
last_layer = in_layer
print(in_layer)
for kernel_shape,out_shape,kernel_stride in zip(conv_kernels,conv_outs,conv_stride):
    last_layer = tn.Conv(kernel_shape,out_shape=out_shape,kernel_stride=kernel_stride,pad=True,activation=tn.ReLU())(last_layer)
    print(last_layer)
for hidden_shape in hidden_shapes:
    last_layer = tn.Dense(hidden_shape,activation=tn.Tanh())(last_layer)
    print(last_layer)
out_layer = tn.Dense(output_shape,activation=tn.Sigmoid())(last_layer)
print(out_layer)

s = tn.System(inputs=[in_layer],outputs=[out_layer])

Input :: [] -> [(32, 32)]
Conv :: [(32, 32)] -> [(10, 10, 4)]
Dense :: [(10, 10, 4)] -> [(26,)]
Conv :: [(32, 32)] -> [(10, 10, 4)] idx: 1 => Dense :: [(10, 10, 4)] -> [(26,)] idx: 0
Input :: [] -> [(32, 32)] idx: 2 => Conv :: [(32, 32)] -> [(10, 10, 4)] idx: 1


In [5]:
import multiprocessing
import functools

def batch(length=1000):    
    total,failures = 0,0
    state = None
    for true_out,input in ocr_data.tagged_2d_data(length):
        guess_out,state = s.guess([input],return_state=True)
        #print(guess_out[0]-true_out)
        if np.argmax(guess_out[0]) != np.argmax(true_out):
            failures += 1
        s.learn(state,[true_out],scale=1e-2)
        total += 1
    return total,failures

In [6]:
print('loading weights')
#s.load_weights('OCR_conv_network.h5')

loading weights


In [7]:
#%prun batch(1000)

In [ ]:
try:
    batch_size = 10000
    while True:
        cases,failures = batch(batch_size)
        print('saving weights')
        s.save_weights('OCR_conv_network.h5')
        print('batch accuracy',cases-failures,'/',cases)
        if (cases-failures)/cases > 0.999:
            break
except KeyboardInterrupt:
    print('Stopped by user')

saving weights
batch accuracy 8134 / 10000
saving weights
batch accuracy 9229 / 10000
saving weights
batch accuracy 9295 / 10000
saving weights
batch accuracy 9374 / 10000
saving weights
batch accuracy 9362 / 10000
saving weights
batch accuracy 9433 / 10000
saving weights
batch accuracy 9526 / 10000
saving weights
batch accuracy 9513 / 10000
saving weights
batch accuracy 9544 / 10000
saving weights
batch accuracy 9531 / 10000
saving weights
batch accuracy 9579 / 10000
saving weights
batch accuracy 9589 / 10000
saving weights
batch accuracy 9612 / 10000
saving weights
batch accuracy 9582 / 10000
saving weights
batch accuracy 9578 / 10000
saving weights
batch accuracy 9615 / 10000
saving weights
batch accuracy 9605 / 10000
saving weights
batch accuracy 9634 / 10000
saving weights
batch accuracy 9649 / 10000
saving weights
batch accuracy 9661 / 10000
saving weights
batch accuracy 9665 / 10000
saving weights
batch accuracy 9698 / 10000
saving weights
batch accuracy 9710 / 10000
saving weig

In [ ]:
#for l in s.parts[0].layer:
#    plt.imshow(l.weights[1:].reshape((4,4)))
#    plt.colorbar()
#    plt.show()
#    plt.close()

In [ ]:
#for l in s.parts[1].layer:
#    plt.imshow(l.weights[1:].reshape((32,32)))
#    plt.colorbar()
#    plt.show()
#    plt.close()

In [ ]:
for i,c in enumerate(range(ord('A'),ord('Z'))):
    for j in range(4):
        print('neuron',chr(c),'conv',j)
        plt.imshow(s.parts[-3].layer[0].weights[i,1:].reshape((10,10,4))[:,:,j])
        plt.colorbar()
        plt.show()
        plt.close()

In [ ]:
for i in range(4):
    print('conv kernel',i)
    plt.imshow(s.parts[-2].layer[0].weights[i,1:].reshape((5,5)))
    plt.colorbar()
    plt.show()
    plt.close()